# 토치텍스트 튜토리얼(Torchtext tutorial) - 영어

토치텍스트(Torchtext): 텍스트에 대한 여러 추상화 기능을 제공하는 자연어처리 라이브러리

토치텍스트가 제공하는 기능들
- 파일 로드하기(File Loading) : 다양한 포맷의 코퍼스를 로드합니다.
- 토큰화(Tokenization) : 문장을 단어 단위로 분리해줍니다.
- 단어 집합(Vocab) : 단어 집합을 만듭니다.
- 정수 인코딩(Integer encoding) : 전체 코퍼스의 단어들을 각각의 고유한 정수로 맵핑합니다.
- 단어 벡터(Word Vector) : 단어 집합의 단어들에 고유한 임베딩 벡터를 만들어줍니다. 랜덤값으로 초기화한 값일 수도 있고, 사전 훈련된 임베딩 벡터들을 로드할 수도 있습니다.
- 배치화(Batching) : 훈련 샘플들의 배치를 만들어줍니다. 이 과정에서 패딩 작업(Padding)도 이루어집니다.

토치텍스트가 모든 전처리를 해결해주지는 않습니다. 위 모든 과정 이전에 훈련 데이터, 검증 데이터, 테스트 데이터를 분리하는 작업은 별도로 해주어야 하며 위 모든 과정 이후에 각 샘플에 대해서 단어들을 임베딩 벡터로 맵핑해주는 작업도 별도로 해주어야 합니다.

## 훈련 데이터와 테스트 데이터로 분리하기

In [2]:
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1') # IMDB 리뷰 데이터는 영화 리뷰가 긍정인지 부정인지에 대한 데이터
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [3]:
print('전체 샘플의 개수 : {}'.format(len(df)))

전체 샘플의 개수 : 50000


In [4]:
train_df = df[:25000] # 훈련데이터 25000개
test_df = df[25000:] # 테스트데이터 25000개

In [5]:
train_df.to_csv("train_data.csv", index=False) # 인덱스 저장 x
test_df.to_csv("test_data.csv", index=False) # 인덱스 저장 x

## 필드 정의하기(torchtext.data)

torchtext.data에는 필드(Field)라는 도구를 제공합니다. 필드를 통해 앞으로 어떤 토큰화 전처리를 할 것인지를 정의합니다.

- sequential : 시퀀스 데이터 여부. (True가 기본값)
- use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
- tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
- lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
- batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
- is_target : 레이블 데이터 여부. (False가 기본값)
- fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.

In [2]:
from torchtext.legacy import data # torchtext.data 임포트

# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

OSError: [WinError 127] 지정된 프로시저를 찾을 수 없습니다

## 데이터셋 만들기

TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행합니다.

- path : 파일이 위치한 경로.
- format : 데이터의 포맷.
- fields : 위에서 정의한 필드를 지정. 첫번째 원소는 데이터 셋 내에서 해당 필드를 호칭할 이름, 두번째 원소는 지정할 필드.
- skip_header : 데이터의 첫번째 줄은 무시.

In [ ]:
from torchtext.data import TabularDataset

train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

In [ ]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

In [ ]:
print(vars(train_data[0])) # vars()를 통해서 주어진 인덱스의 샘플을 확인

## 단어 집합(Vocabulary) 만들기

토큰화 전처리(완료) -> 단어 집합(현재) -> 정수 인코딩

정의한 필드에 .build_vocab() 도구를 사용하면 단어 집합을 생성합니다.

- min_freq : 단어 집합에 추가 시 단어의 최소 등장 빈도 조건을 추가.
- max_size : 단어 집합의 최대 크기를 지정.(사실 토치텍스트가 임의로 특별 토큰인 unk(단어 집합에 없는 단어)와 pad(패딩)를 추가해서 max_size+2가 된다)

In [ ]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [ ]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

In [ ]:
print(TEXT.vocab.stoi) # .stoi를 통해 단어 집합 내의 단어들 확인

## 토치텍스트의 데이터로더 만들기

토큰화 전처리(완료) -> 단어 집합(완료) -> 정수 인코딩(현재)

데이터로더는 데이터셋에서 미니 배치만큼 데이터를 로드하게 만들어주는 역할을 합니다. 토치텍스트에서는 Iterator를 사용하여 데이터로더를 만듭니다. 각 샘플은 더 이상 단어 시퀀스가 아니라 정수 시퀀스임을 볼 수 있습니다.

In [4]:
from torchtext.data import Iterator

batch_size = 5

train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

OSError: [WinError 127] 지정된 프로시저를 찾을 수 없습니다

In [ ]:
batch = next(iter(train_loader)) # 첫번째 미니배치

In [ ]:
print(type(batch)) # 일반적인 데이터로더는 미니 배치를 텐서로 가져오지만 토치텍스트의 데이터로더는 torchtext.data.batch.Batch 객체를 가져온다

In [ ]:
print(batch.text) #  실제 데이터 텐서에 접근하기 위해서는 정의한 필드명을 사용해야 한다다